# CNN

In [1]:
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
import matplotlib.pyplot as plt
tf.compat.v1.set_random_seed(0)
from tensorflow import keras
import numpy as np
np.random.seed(0)
import itertools
from sklearn.metrics import precision_score, accuracy_score, recall_score, confusion_matrix, ConfusionMatrixDisplay

In [2]:

train_dir='/Users/hiten/Documents/Maincode/Dataset/train'
valid_dir='/Users/hiten/Documents/Maincode/Dataset/val'
test_dir='/Users/hiten/Documents/Maincode/Dataset/test'

Size=224

In [3]:
train_gen = tf.keras.utils.image_dataset_from_directory(directory=train_dir,
                                         image_size=(256, 256),
                                         batch_size = 32,
                                         )
test_gen = tf.keras.utils.image_dataset_from_directory(directory=valid_dir,
                                        image_size=(256, 256),
                                        batch_size = 32,
                                        )

rescale = tf.keras.layers.Rescaling(scale=1.0/255)
train_gen = train_gen.map(lambda image,label:(rescale(image),label))
test_gen  = test_gen.map(lambda image,label:(rescale(image),label))

test = tf.keras.utils.image_dataset_from_directory(directory=test_dir,
                                        image_size=(256, 256))
test  = test.map(lambda image,label:(rescale(image),label))

Found 42230 files belonging to 38 classes.
Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2022-09-09 00:17:38.969907: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-09-09 00:17:38.970449: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Found 9042 files belonging to 38 classes.
Found 9071 files belonging to 38 classes.


In [4]:
new_model = tf.keras.models.load_model(f'content/saved_model/Best_CNN_new_4th_sep/')

In [5]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_gen)

2022-09-09 00:18:07.817228: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-09-09 00:18:07.942968: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


283/283 [==============================] - 52s 180ms/step - loss: 0.1180 - accuracy: 0.9718


In [6]:
labels = []
predictions = []
for x,y in test_gen:
    labels.append(list(y.numpy()))
    predictions.append(tf.argmax(new_model.predict(x),1).numpy())

1/1 [==============================] - 0s 164ms/step


2022-09-09 00:18:59.758898: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 142ms/step


2022-09-09 00:20:01.722169: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [7]:
predictions = list(itertools.chain.from_iterable(predictions))
labels = list(itertools.chain.from_iterable(labels))

In [8]:
from sklearn.metrics import classification_report,cohen_kappa_score,matthews_corrcoef
print(f"Model Best CNN \n")
print(classification_report(labels, predictions))

cohen_kappa_score = cohen_kappa_score(labels, predictions)
Metthews_correlation = matthews_corrcoef(labels, predictions)

print("Cohen Kappa Score: ", cohen_kappa_score)
print("Matthews correlation coefficient : ", Metthews_correlation)

Model Best CNN 

              precision    recall  f1-score   support

           0       0.90      0.90      0.90       150
           1       0.96      0.98      0.97       150
           2       0.99      0.99      0.99       150
           3       0.97      0.97      0.97       246
           4       0.98      0.99      0.98       225
           5       0.98      1.00      0.99       157
           6       1.00      1.00      1.00       150
           7       0.87      0.89      0.88       150
           8       1.00      1.00      1.00       178
           9       0.88      0.85      0.86       150
          10       0.99      1.00      1.00       174
          11       0.99      0.96      0.97       177
          12       1.00      1.00      1.00       207
          13       0.96      1.00      0.98       161
          14       0.98      0.97      0.97       150
          15       1.00      0.99      1.00       826
          16       0.98      0.96      0.97       344
          

In [9]:
def counts_from_confusion(confusion):
    """
    Obtain TP, FN FP, and TN for each class in the confusion matrix
    """

    counts_list = []

    # Iterate through classes and store the counts
    for i in range(confusion.shape[0]):
        tp = confusion[i, i]

        fn_mask = np.zeros(confusion.shape)
        fn_mask[i, :] = 1
        fn_mask[i, i] = 0
        fn = np.sum(np.multiply(confusion, fn_mask))

        fp_mask = np.zeros(confusion.shape)
        fp_mask[:, i] = 1
        fp_mask[i, i] = 0
        fp = np.sum(np.multiply(confusion, fp_mask))

        tn_mask = 1 - (fn_mask + fp_mask)
        tn_mask[i, i] = 0
        tn = np.sum(np.multiply(confusion, tn_mask))

        counts_list.append({'Class': i,
                            'TP': tp,
                            'FN': fn,
                            'FP': fp,
                            'TN': tn})

    return counts_list

In [10]:
cm = confusion_matrix(labels, predictions)
counting_cm_list=counts_from_confusion(cm)
counting_cm_list

[{'Class': 0, 'TP': 135, 'FN': 15.0, 'FP': 15.0, 'TN': 8877.0},
 {'Class': 1, 'TP': 147, 'FN': 3.0, 'FP': 6.0, 'TN': 8886.0},
 {'Class': 2, 'TP': 149, 'FN': 1.0, 'FP': 1.0, 'TN': 8891.0},
 {'Class': 3, 'TP': 239, 'FN': 7.0, 'FP': 8.0, 'TN': 8788.0},
 {'Class': 4, 'TP': 222, 'FN': 3.0, 'FP': 5.0, 'TN': 8812.0},
 {'Class': 5, 'TP': 157, 'FN': 0.0, 'FP': 4.0, 'TN': 8881.0},
 {'Class': 6, 'TP': 150, 'FN': 0.0, 'FP': 0.0, 'TN': 8892.0},
 {'Class': 7, 'TP': 134, 'FN': 16.0, 'FP': 20.0, 'TN': 8872.0},
 {'Class': 8, 'TP': 178, 'FN': 0.0, 'FP': 0.0, 'TN': 8864.0},
 {'Class': 9, 'TP': 127, 'FN': 23.0, 'FP': 18.0, 'TN': 8874.0},
 {'Class': 10, 'TP': 174, 'FN': 0.0, 'FP': 1.0, 'TN': 8867.0},
 {'Class': 11, 'TP': 170, 'FN': 7.0, 'FP': 2.0, 'TN': 8863.0},
 {'Class': 12, 'TP': 206, 'FN': 1.0, 'FP': 1.0, 'TN': 8834.0},
 {'Class': 13, 'TP': 161, 'FN': 0.0, 'FP': 6.0, 'TN': 8875.0},
 {'Class': 14, 'TP': 145, 'FN': 5.0, 'FP': 3.0, 'TN': 8889.0},
 {'Class': 15, 'TP': 821, 'FN': 5.0, 'FP': 0.0, 'TN': 8216.

In [11]:
import pandas as pd
from tqdm.notebook import tqdm
Sensitivity = []
Specificity = []
class_accuracy =[]
class_label = []
Recall = []
Precision = []
F1_score = []

for i in tqdm(range(38)):
    tp = counting_cm_list[i]['TP']
    tn = counting_cm_list[i]['TN']
    fp = counting_cm_list[i]['FP']
    fn = counting_cm_list[i]['FN']
    
    
    # sensitivity
    senti = (tp / (tp + fn))*100
    senti = "{:.2f}".format(senti)
    #specificity
    speci = (tn / (tn + fp))*100
    speci = "{:.2f}".format(speci)
    
    #Precision
    precision_new = "{:.2f}".format((tp/(tp+fp))*100)
    #recall
    recall_new = "{:.2f}".format((tp/(tp+fn))*100)
    #F1-Score
    
    f1_score_new = 2*(float(precision_new)*float(recall_new))/(float(recall_new)+float(precision_new))
    f1_score_new = "{:.2f}".format(f1_score_new)
    
    # Overall accuracy
    ACC = ((tp + tn)/(tp+fp+fn+tn))*100
    ACC = "{:.2f}".format(ACC)
    class_label.append(i)
    class_accuracy.append(ACC)
    Sensitivity.append(senti)
    Specificity.append(speci)
    Precision.append(precision_new)
    Recall.append(recall_new)
    F1_score.append(f1_score_new)
    


  0%|          | 0/38 [00:00<?, ?it/s]

In [12]:

recall = np.diag(cm) / np.sum(cm, axis = 1)
precision = np.diag(cm) / np.sum(cm, axis = 0)


total_specificity = [float(i) for i in Specificity]
total_specificity = np.mean(total_specificity)
print("Specificity: ",total_specificity,"%")

total_Sensitivity = [float(i) for i in Sensitivity]
total_Sensitivity = np.mean(total_Sensitivity)
print("Sensitivity: ",total_Sensitivity,"%")

total_recall = np.mean(recall)
total_precision = np.mean(precision)
total_f1_score = 2*(total_precision*total_recall)/(total_precision+total_recall)

print("Recall: ", "{:.2f}".format(total_recall*100),"%")
print("Precision: ", "{:.2f}".format(total_precision*100),"%")
print("F1-Score: ", "{:.2f}".format(total_f1_score*100),"%")



Specificity:  99.92368421052632 %
Sensitivity:  96.58736842105263 %
Recall:  96.59 %
Precision:  96.44 %
F1-Score:  96.51 %


In [13]:
df = pd.DataFrame()
df["Class_Label"] = class_label
df["Specificity(%)"] = Specificity
df["Sensitivity(%)"] = Sensitivity
df["Precision(%)"] = Precision
df["Recall(%)"] = Recall
df["F1_score(%)"] = F1_score
df["Accuracy(%)"] = class_accuracy

df2 = pd.DataFrame({'Class_Label': ['Total scores'],
                    'Specificity(%)':[total_specificity],
                    'Sensitivity(%)':[total_Sensitivity],
                    'Precision(%)':[total_precision],
                    'Recall(%)':[total_recall],
                    'F1_score(%)':[total_f1_score],
                    'Accuracy(%)':[acc],
                    
                    })

df3 = pd.DataFrame({'Class_Label': ['Total scores'],
                    'Specificity(%)':[total_specificity],
                    'Sensitivity(%)':[total_Sensitivity],
                    'Precision(%)':[total_precision],
                    'Recall(%)':[total_recall],
                    'F1_score(%)':[total_f1_score],
                    'Accuracy(%)':[acc],
                    
                    })
df = pd.concat([df, df2], ignore_index = True, axis = 0)

In [14]:
df

,Class_Label,Specificity(%),Sensitivity(%),Precision(%),Recall(%),F1_score(%),Accuracy(%)
0,0,99.83,90.00,90.00,90.00,90.00,99.67
1,1,99.93,98.00,96.08,98.00,97.03,99.90
2,2,99.99,99.33,99.33,99.33,99.33,99.98
3,3,99.91,97.15,96.76,97.15,96.95,99.83
4,4,99.94,98.67,97.80,98.67,98.23,99.91
5,5,99.95,100.00,97.52,100.00,98.74,99.96
6,6,100.00,100.00,100.00,100.00,100.00,100.00
7,7,99.78,89.33,87.01,89.33,88.15,99.60
8,8,100.00,100.00,100.00,100.00,100.00,100.00
9,9,99.80,84.67,87.59,84.67,86.11,99.55


In [15]:
df.to_csv('CSV_files/best_CNN.csv')